In [68]:
import pandas as pd
import regex
from pathlib import Path

In [69]:
# Excel Data Sets
tx_centers2020 = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\hrsa\ORG_OTC_FCT_DET.csv')
tx_centers2009 = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\txcenters2009clean.csv')
tx_centers2009 = tx_centers2009.replace(r"^ +| +$", r"", regex=True)
# # Old CSV. A cleaned version is available now.
# tx_centers2009 = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\txcenters2009list.csv')

In [70]:
# This dataset was created 5-27-2021 by HRSA
# tx_center Count - 254
print(tx_centers2020['OTC Name'].nunique())
# State Count - 39
print(tx_centers2020.State.nunique())
# Cities Count - 58
print(tx_centers2020.City.nunique())

print(tx_centers2020.info())

254
39
58
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 31 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   OPO Name                                               809 non-null    object 
 1   OTC Name                                               809 non-null    object 
 2   OTC Service List                                       809 non-null    object 
 3   OPO Provider #                                         809 non-null    object 
 4   Address                                                809 non-null    object 
 5   City                                                   809 non-null    object 
 6   State                                                  809 non-null    object 
 7   ZIP                                                    809 non-null    object 
 8   OPO Telephone #                         

In [71]:
# This dataset was created 5-2009 by SRTR
# tx_center Count - 327
print(tx_centers2009['OTC Name'].nunique())
# State Count - 48
print(tx_centers2009.State.nunique())
# Cities Count - 169
print(tx_centers2009.City.nunique())

print(tx_centers2009.info())

327
48
169
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   OTC Name  333 non-null    object
 1   City      333 non-null    object
 2   State     333 non-null    object
dtypes: object(3)
memory usage: 7.9+ KB
None


In [72]:
tx_centers2020.head()

,OPO Name,OTC Name,OTC Service List,OPO Provider #,Address,City,State,ZIP,OPO Telephone #,OTC Code,...,Common City Name with State Abbreviation,Common Congressional District Name,Common Congress Number Text,Common State FIPS Code Congressional District Number,Common Postal Code,Longitude,Latitude,Data Warehouse Record Create Date,Data Warehouse Record Create Date Text,Unnamed: 30
0,Alabama Organ Center,Children's of Alabama,"Heart, Kidney, Liver",01P001,502 20th St S,Birmingham,AL,35233-2028,205-731-9200,ALCH,...,"Birmingham, AL",Alabama District 07,115,107,35233,-86.804823,33.50352,05/27/2021,2021/05/27,NaN
1,Alabama Organ Center,Children's of Alabama,"Heart, Kidney, Liver",01P001,502 20th St S,Birmingham,AL,35233-2028,205-731-9200,ALCH,...,"Birmingham, AL",Alabama District 07,115,107,35233,-86.804823,33.50352,05/27/2021,2021/05/27,NaN
2,Alabama Organ Center,Children's of Alabama,"Heart, Kidney, Liver",01P001,502 20th St S,Birmingham,AL,35233-2028,205-731-9200,ALCH,...,"Birmingham, AL",Alabama District 07,115,107,35233,-86.804823,33.50352,05/27/2021,2021/05/27,NaN
3,Alabama Organ Center,University of Alabama Hospital,"Heart, Intestine, Kidney, Liver, Lung, Pancre...",01P001,502 20th St S,Birmingham,AL,35233-2028,205-731-9200,ALUA,...,"Birmingham, AL",Alabama District 07,115,107,35205,-86.814637,33.48393,05/27/2021,2021/05/27,NaN
4,Alabama Organ Center,University of Alabama Hospital,"Heart, Intestine, Kidney, Liver, Lung, Pancre...",01P001,502 20th St S,Birmingham,AL,35233-2028,205-731-9200,ALUA,...,"Birmingham, AL",Alabama District 07,115,107,35205,-86.814637,33.48393,05/27/2021,2021/05/27,NaN


In [73]:
tx_centers2009.head()

,OTC Name,City,State
0,Baptist Medical Center Princeton,Birmingham,AL
1,Children's Hospital of Alabama,Birmingham,AL
2,University of Alabama Hospital,Birmingham,AL
3,University of South Alabama Medical Center,Mobile,AL
4,Baptist Medical Center,Little Rock,AR


In [74]:
# # This work is finished. New CSV for all of it is cleaned up and included in current code.

# # tx_centers2009 = tx_centers2009.ALABAMA.str.split(',', expand=True)

# # This did not work.
# from pathlib import Path
# filepath = Path('C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\test.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# tx_centers2009.to_csv(filepath)

# # This line is current, run it here if I have to redo the work.
#tx_centers2009 = tx_centers2009.replace(r"^ +| +$", r"", regex=True)

# # This worked
# compression_opts = dict(method='zip',
#                         archive_name='out.csv')  
# tx_centers2009.to_csv('out.zip', index=False,
#           compression=compression_opts)

In [75]:
# Questions:
# Do surgeons work at different TX Centers?
# Get Footprints of the OPOs

# Match up the 2009 and 2021 lists
# Get the 2021 list noted with NEW hospitals.
# Get the 2009 list noted with CLOSURES
# Get the 2009 list coded with lat.long